In [0]:
%fs ls 's3://nokia-assignment-s3/assignment_data/raw_xml_data/'

In [0]:
SHOW CATALOGS;

In [0]:
CREATE CATALOG IF NOT EXISTS `nokia-assginment-catalog`
MANAGED LOCATION 's3://nokia-assignment-s3/'
COMMENT 'this is a catalog for nokia assginment';

In [0]:
SHOW CATALOGS;

In [0]:
select current_catalog();

In [0]:
USE CATALOG `nokia-assginment-catalog`;

In [0]:
select current_catalog();

In [0]:
CREATE SCHEMA IF NOT EXISTS assignment_data
MANAGED LOCATION 's3://nokia-assignment-s3/assignment_data/';

In [0]:
SHOW SCHEMAS;

In [0]:
CREATE SCHEMA IF NOT EXISTS bronze
MANAGED LOCATION 's3://nokia-assignment-s3/assignment_data/bronze/';

CREATE SCHEMA IF NOT EXISTS silver
MANAGED LOCATION 's3://nokia-assignment-s3/assignment_data/silver/';

CREATE SCHEMA IF NOT EXISTS gold
MANAGED LOCATION 's3://nokia-assignment-s3/assignment_data/gold/';

In [0]:
USE SCHEMA assignment_data;

In [0]:
CREATE EXTERNAL VOLUME xml_raw_data LOCATION 's3://nokia-assignment-s3/assignment_data/raw_xml_data/';


In [0]:
%fs ls '/Volumes/nokia-assginment-catalog/assignment_data/xml_raw_data'

In [0]:
CREATE EXTERNAL VOLUME bronze_volume LOCATION 's3://nokia-assignment-s3/assignment_data/bronze/';


In [0]:
%python
from pyspark.sql import SparkSession

# Define catalog and schema
catalog = "`nokia-assginment-catalog`"  # Backticks handle special characters
schema = "gold"

# Create Spark session
spark = SparkSession.builder.getOrCreate()

# Query to list volumes
query = f"SHOW VOLUMES IN {catalog}.{schema}"
volumes_df = spark.sql(query)

# Extract volume names
volumes = [row["volume_name"] for row in volumes_df.collect()]

# Print volumes before deletion
print("Volumes found:", volumes)

# Loop through each volume and delete files inside
for volume in volumes:
    volume_path = f"/Volumes/{catalog}/{schema}/{volume}"
    try:
        print(f"📂 Listing files in {volume_path}...")
        files = dbutils.fs.ls(volume_path)

        for file in files:
            print(f"🗑 Deleting: {file.path}")
            dbutils.fs.rm(file.path, recurse=True)

        print(f"✅ Successfully cleaned {volume_path}")
    except Exception as e:
        print(f"❌ Failed to clean {volume_path}: {e}")

print("All volumes cleaned.")


In [0]:
%python
from pyspark.sql import SparkSession

# Define catalog and schema
catalog = "`nokia-assginment-catalog`"  # Backticks handle special characters
schema = "gold"

# Create Spark session
spark = SparkSession.builder.getOrCreate()

# List of volumes to recreate (use the same ones you deleted)
volumes = [
    "applicants",
    "applicants_individual",
    "claims",
    "claims_individual",
    "complete_patents",
    "inventors",
    "inventors_individual",
    "ipc_classifications",
    "ipc_individual",
    "patents",
    "us_applicants",
    "us_applicants_individual"
]

# Loop through and create volumes
for volume in volumes:
    create_query = f"CREATE VOLUME {catalog}.{schema}.{volume}"
    try:
        spark.sql(create_query)
        print(f"✅ Created volume: {volume}")
    except Exception as e:
        print(f"❌ Failed to create volume {volume}: {e}")

print("🎉 All volumes recreated successfully.")


In [0]:
DROP SCHEMA IF EXISTS `nokia-assginment-catalog`.patent_data cascade;